# Project 1 – Charles – Metrics

In [10]:
import numpy as np
import pandas as pd
import re
import matplotlib
import seaborn as sns

# Adding Session ID

In [13]:
# do not run this entire section if your dataset already has unique ID

In [11]:
data = pd.read_csv("Cleaned_Data.csv", low_memory=False)
df = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
df['ae_session_length'] = df['ae_session_length'].fillna('-1')
df['ae_session_length'] = df['ae_session_length'].str.replace(',', '')
df['ae_session_length'] = df['ae_session_length'].astype(float)
df['time'] = df['time'] = pd.to_datetime(df['time'])
df

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,...,ios_lib_version,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown
165943,165943,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,...,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0
183940,183940,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,...,NaN,NaN,NaN,NaN,INTL,0,0,0,0,0
40790,40790,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1,-1,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
40792,40792,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1,-1,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
40793,40793,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1,-1,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120809,120809,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,-1,-1,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
120810,120810,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,-1,-1,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
120811,120811,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,-1,-1,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0
77301,77301,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,-1,-1,...,NaN,NaN,NaN,NaN,US,1,0,0,0,0


In [12]:
def add_session_id(df):
    df['session_id'] = -1
    sessions = df[['name_x', 'distinct_id', 'time', 'ae_session_length']].loc[df['name_x'] == '$ae_session']
    sessions['start'] = sessions['time'] - np.array([pd.Timedelta(seconds=np.ceil(x)) for x in sessions['ae_session_length']])
    curr_id = 1
    for index, row in sessions.iterrows():
        df.loc[(df['session_id'] == -1) 
               & (df['time'] <= row['time']) 
               & (df['time'] >= row['start']) 
               & (df['distinct_id'] == row['distinct_id']), 'session_id'] = curr_id
        curr_id += 1

add_session_id(df)
df
    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/arrays/datetimelike.py:1268: PerformanceWarning: Adding/subtracting array of DateOffsets to DatetimeArray not vectorized
  PerformanceWarning,


,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,...,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id
165943,165943,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
183940,183940,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
40790,40790,AppOpen,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:32,-1.0,Palo Alto,California,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,1
40792,40792,Visit Signup/Login Page,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:48:35,-1.0,Palo Alto,California,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,1
40793,40793,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:49:02,30.1,Palo Alto,California,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120809,120809,Feed - Scroll Down,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:23,-1.0,Portland,Oregon,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,29813
120810,120810,Profile - Change Section,xbarbarazhong@gmail.com_6704,2019-12-12 14:29:28,-1.0,Portland,Oregon,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,29813
120811,120811,$ae_session,xbarbarazhong@gmail.com_6704,2019-12-12 14:30:54,190.4,Portland,Oregon,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,29813
77301,77301,AppOpen,yoon.s.jeong@gmail.com_6485,2019-10-09 11:12:29,-1.0,Chicago,Illinois,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,29814


In [14]:
df.loc[df['session_id'] == -1]

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,...,ae_first_app_open_date,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id
165943,165943,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-02-13 15:52:50,-1.0,Mumbai,Maharashtra,IN,-1,-1,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
183940,183940,$ae_first_open,00000000-0000-0000-0000-000000000000,2020-03-08 20:43:54,-1.0,Ahmedabad,Gujarat,IN,-1,-1,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
4537,4537,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-04 16:28:25,-1.0,Austin,Texas,US,1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
6133,6133,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-06 04:02:51,-1.0,San Antonio,Texas,US,1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
10757,10757,AppOpen,001210DC-54C8-43AD-B295-148F47818391,2019-08-09 15:45:25,-1.0,Austin,Texas,US,1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28614,28614,Sidebar - Sign Out,virtrutest99@gmail.com_5848,2019-08-19 20:54:49,-1.0,Delhi,National Capital Territory of Delhi,IN,-1,-1,...,NaN,NaN,NaN,INTL,0,0,0,0,0,-1
158372,158372,AppOpen,willwojt@gmail.com_6868,2020-02-03 12:19:01,-1.0,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
31856,31856,$ae_updated,wolphramite@gmail.com_291,2019-08-20 01:42:41,-1.0,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,NaN,US,1,0,0,0,0,-1
95802,95802,$campaign_delivery,wolphramite@gmail.com_291,2019-10-31 07:28:16,-1.0,NaN,NaN,NaN,-1,-1,...,NaN,NaN,NaN,NaN,2,0,0,0,0,-1


In [15]:
df.to_csv("Cleaned_Dataset_With_ID.csv")

# Adding Time Between Sessions

In [6]:
data = pd.read_csv("Cleaned_Dataset_With_ID.csv", low_memory=False)

In [7]:
data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [9]:
data = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
data.columns

Index(['name_x', 'distinct_id', 'time', 'ae_session_length', 'city', 'region',
       'mp_country_code', 'locationSetting', 'notificationSettings',
       'FirstTimeUploaded', 'Signup With', 'imageAdded', 'commentAdded',
       'commentLength', 'ImageUploaded', 'ae_total_app_session_length',
       'name_y', 'ae_total_app_sessions', 'timezone', 'last_seen',
       'ios_app_version', 'ios_version', 'ios_device_model', 'ios_lib_version',
       'ae_first_app_open_date', 'active_timespan', 'average_session_time',
       'country', 'US', 'Unknown', 'session_id'],
      dtype='object')